In [1]:
import numpy as np


from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt #Para graficar
from skimage.io import imshow #Para graficar las imagenes
import tensorflow as tf


import os
import glob
##from PIL import Image
import cv2

#import pyheif

import matplotlib.pyplot as plt

In [2]:
#root_path = "C:/MaestriaDocs/DMA - Proyecto/caras_grupo4/"
root_path = "../"

directorio_personas = root_path + "Caras_cortadas"

Obtenemos de las carpetas originales las fotos

In [3]:
from PCA_funciones import matriz_fotos_desde_carpeta

image_matrix, image_person = matriz_fotos_desde_carpeta(directorio_personas)

Carpeta: Abel
Carpeta: Carlos
Carpeta: Federico G
Carpeta: Federico R
Carpeta: Florencia
Carpeta: Franco A
Carpeta: Franco S
Carpeta: Gerard
Carpeta: Gustavo
Carpeta: Joaquin
Carpeta: Juan
Carpeta: Lautaro
Carpeta: Lisandro
Carpeta: Marco
Carpeta: Matias
Carpeta: Natalia
Carpeta: Noelia
Carpeta: Paola
Carpeta: Victorio


## Aplicamos Backpropagation
Transformamos las imagenes, con la cantidad de copoenentes 

In [4]:
from sklearn.decomposition import PCA
import pickle

# Cargar los componentes PCA guardados
ruta_pca = root_path + "PCA/Componentes_pca.npy" #lo tenemos en memoria pero lo leemos nuevamente por si separamos el codigo
pca_components = np.load(ruta_pca)

# Cargar el escalador
ruta_scaler = root_path + 'PCA/scaler.pkl' #
with open(ruta_scaler, 'rb') as f:
    scaler = pickle.load(f)

# Función para transformar una imagen con un rango específico de componentes PCA
def transformar_imagen_pca(imagen, scaler, pca_components, num_componentes=50, inicio_componente=0):
    # Estandarizar la imagen
    imagen_estandarizada = scaler.transform(imagen)
    
    # Seleccionar el rango de componentes
    componentes_seleccionados = pca_components[inicio_componente:inicio_componente + num_componentes]
    
    # Realizar la proyección manualmente
    imagen_pca = np.dot(imagen_estandarizada, componentes_seleccionados.T)
    return imagen_pca


# Especificar la cantidad de componentes a utilizar y el componente de inicio
num_componentes = 60
inicio_componente = 2 #inicia a partir de la 3ta componente

# Transformar la nueva imagen utilizando los componentes especificados
imagenes_pca_reducidas = transformar_imagen_pca( image_matrix, scaler, pca_components, num_componentes, inicio_componente)

print(imagenes_pca_reducidas)

imagenes_pca_reducidas.shape
personas = np.unique(image_person)
image_person #(vector que tiene el nombre de la persona de cada foto)
personas
personas.shape
cantidad_personas = len(personas)
print(cantidad_personas) 
cantidad_fotos = len(image_person)
cantidad_fotos
# salida_lista = []  # Array vacío para almacenar las listas de 0 y 1
# cantidad_personas = len(personas) #podiramos hacer un elemento menos pero por ahora lo dejamos en el total
# # Recorrer cada elemento en "image_person"
# for imagen_persona in image_person:
#   lista_persona = [0] * cantidad_personas   # Inicializar lista con 18 ceros
  
#   # Buscar el índice del nombre de la imagen en "personas"
#  # Encontrar el índice de la persona
#   indice_persona =  np.where(personas == imagen_persona)[0]

#   # Si se encuentra la persona, actualizar la lista
#   if len(indice_persona) > 0:
#     lista_persona[indice_persona[0]] = 1
  
#   # Actualizar el elemento correspondiente en la lista con 1

  
#   # Agregar la lista a Y
#   salida_lista.append(lista_persona)

# salida_lista = np.array(salida_lista) 
# salida_lista

imagenes_pca_reducidas[0]
X=imagenes_pca_reducidas
X.shape
from sklearn.preprocessing import  LabelBinarizer
# Convertir los valores de la var categórica en números
#salida_lista = LabelBinarizer().fit_transform(image_person) #convierte los nombres (var categorica) en vextores con un 1 en la posicion del nombre
#correspondiente, tiene un renglon por foto y el vector una columna por nombre solo un 1 en el nombre de la persona de la foto


label_binarizer = LabelBinarizer()
salida_lista = label_binarizer.fit_transform(image_person)
salida_lista
salida_lista.shape
X = imagenes_pca_reducidas
Y = salida_lista

# Normalización de las entradas
"""mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - mean) / std
"""
# Convierte la lista a un arreglo NumPy
#salida_lista = nombres_personas
salida = np.array(salida_lista)
Y = salida #salida.reshape(len(X),1)

epoch_limit = 30


# Semilla para reproducibilidad
np.random.seed(1021) #Establece la semilla para las funciones aleatorias de numpy.  
tf.random.set_seed(1021) #Establece la semilla para las funciones aleatorias de TensorFlow, asegurando que los pesos iniciales de la red neuronal y 4cualquier otra operación aleatoria en TensorFlow sean reproducibles.
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1021)

[[ 7.22160860e+00 -7.80849051e+00 -2.07612021e-01 ... -7.16897218e-01
  -2.28025618e-01 -3.38966655e-01]
 [-7.84286624e+00 -1.12024841e+01  4.89598300e-01 ... -7.43509551e-01
  -6.18654161e-01  2.88035939e-01]
 [-6.55630242e+00 -7.80088794e+00 -1.49179444e+00 ... -1.37140978e+00
  -2.96450608e-01  2.05965925e-02]
 ...
 [ 3.55657307e+00  6.49301206e-01 -6.23891347e+00 ... -6.32500626e-02
   3.18870940e-01 -6.26304974e-01]
 [-2.65755973e+00 -4.44534213e-03 -8.82886566e+00 ...  2.92722209e-01
  -9.78650328e-02 -2.65089331e-01]
 [-2.62864833e+00  8.66772857e-01 -8.11920552e+00 ...  1.20877517e-01
  -1.40180893e-02 -1.42284246e-01]]
19


## Back propagation manual

In [5]:
# NUEVO # ------------------------------------------

# backpropagation, just one hidden layer
# lo hago con  matrices de pesos
# puedo tener tantos inputs como quiera
# puedo tener tantas neuronas ocultas como quiera
# puedo tener tanas neuronas de salida como quiera
# fuera de este codigo esta la decision que tomo segun el valor de salida de cada neurona de salida

import math
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.preprocessing import standardize

#from graficos import perceptron_plot 

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + math.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + math.exp(-2.0*x) ) - 1.0
    return y

func_eval_vec = np.vectorize(func_eval)

def deriv_eval(fname, y):  #atencion que y es la entrada y=f( x )
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y*(1.0-y)
        case "tansig":
            d = 1.0 - y*y
    return d

deriv_eval_vec = np.vectorize(deriv_eval)


In [6]:
len(X)

1206

In [7]:
#new_faces_array_transformed.shape #vemos el vector con las caras y sus 60 componentes a partir de la 3ra

# Normalización de las entradas
#mean = np.mean(X, axis=0)
#std = np.std(X, axis=0)
#X = (X - mean) / std

# Convierte la lista a un arreglo NumPy
#salida_lista = nombres_personas
#salida = np.array(salida_lista)
#Y = salida #salida.reshape(len(X),1)

filas_qty = len(X)        #la cantidad de fotos
input_size = X.shape[1]   #  entradas la cantidad de componentes
                            # hidden_size = 3  # neuronas capa oculta
hidden_size1 = filas_qty       # 8  Neuronas en la primera capa oculta, para el 8 teniamos 8 recuadros, aca por ahora ponemos la cantidad de fotos 
hidden_size2 = 3               # Neuronas en la segunda capa oculta
#output_size = Y.shape[2]  # 2 neurona
output_size = cantidad_personas #VEER , vector con la probabilidad q sea la persona de la posicion

# defino las funciones de activacion de cada capa
hidden_FUNC = 'logsig'  # uso la logistica
output_FUNC = 'tansig'  # uso la tangente hiperbolica

# Incializo las matrices de pesos azarosamente
# W1 son los pesos que van del input a la capa oculta
# W2 son los pesos que van de la capa oculta a la capa de salida
np.random.seed(1021) #mi querida random seed para que las corridas sean reproducibles
W1 = np.random.uniform(-0.5, 0.5, [hidden_size1, input_size])
X01 = np.random.uniform(-0.5, 0.5, [hidden_size1, 1] )
W2 = np.random.uniform(-0.5, 0.5, [hidden_size2, hidden_size1])
X02 = np.random.uniform(-0.5, 0.5, [hidden_size2, 1] )
W3 = np.random.uniform(-0.5, 0.5, [output_size, hidden_size2])
X03 = np.random.uniform(-0.5, 0.5, [output_size, 1] )

In [8]:
# # Avanzo la red, forward
# # para TODOS los X al mismo tiempo ! 
# #  @ hace el producto de una matrix por un vector_columna
# hidden_estimulos = W1 @ X.T + X01
# hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
# output_estimulos = W2 @ hidden_salidas + X02
# output_salidas = func_eval_vec(output_FUNC, output_estimulos)


# Función de propagación hacia adelante (forward propagation)
def forward_propagation(X, W1, b1, W2, b2, W3, b3):
    hidden1_estimulos = W1 @ X.T + b1
    hidden1_salidas = func_eval_vec(hidden_FUNC, hidden1_estimulos)
    hidden2_estimulos = W2 @ hidden1_salidas + b2
    hidden2_salidas = func_eval_vec(hidden_FUNC, hidden2_estimulos)
    output_estimulos = W3 @ hidden2_salidas + b3
    output_salidas = func_eval_vec(output_FUNC, output_estimulos)
    return hidden1_salidas, hidden2_salidas, output_salidas, hidden1_estimulos, hidden2_estimulos, output_estimulos


hidden_salida1, hidden_salida2, output_salidas, _, _ ,_ = forward_propagation(X, W1, X01, W2, X02, W3, X03)


# calculo el error promedi general de TODOS los X
Error= np.mean( (Y.T - output_salidas)**2 )

# Inicializo
#epoch_limit = 5000    # para terminar si no converge
Error_umbral = 1.0e-06
#Error_umbral = 1.0e-15
learning_rate = 0.5
#learning_rate = 0.2
Error_last = 10    # lo debo poner algo dist a 0 la primera vez
epoch = 0

while ( math.fabs(Error_last-Error)>Error_umbral and (epoch < epoch_limit)):
    epoch += 1
    Error_last = Error

    # recorro siempre TODA la entrada
    for fila in range(filas_qty): #para cada input x_sub_fila del vector X
        # propagar el x hacia adelante
        # hidden_estimulos = W1 @ X[fila:fila+1, :].T + X01
        # hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
        # output_estimulos = W2 @ hidden_salidas + X02
        # output_salidas = func_eval_vec(output_FUNC, output_estimulos)
        hidden_salida1, hidden_salida2, output_salidas,  _, _ ,_  = forward_propagation(X[fila:fila+1, :], W1, X01, W2, X02, W3, X03)

        # calculo los errores en la capa hidden y la capa output
        ErrorSalida = Y[fila:fila+1,:].T - output_salidas
        # output_delta es un solo numero
        output_delta = ErrorSalida * deriv_eval_vec(output_FUNC, output_salidas)
        
        # hidden_delta es un vector columna
        # hidden_delta = deriv_eval_vec(hidden_FUNC, hidden_salidas)*(W2.T @ output_delta)
        # hidden_delta es un vector columna
        hidden_delta2 = deriv_eval_vec(hidden_FUNC, hidden_salida2)*(W3.T @ output_delta)

        # hidden_delta es un vector columna
        hidden_delta1 = deriv_eval_vec(hidden_FUNC, hidden_salida1)*(W2.T @ hidden_delta2)

        # ya tengo los errores que comete cada capa
        # corregir matrices de pesos, voy hacia atras
        # backpropagation
        W1 = W1 + learning_rate * (hidden_delta1 @ X[fila:fila+1, :] )
        X01 = X01 + learning_rate * hidden_delta1
        W2 = W2 + learning_rate * (hidden_delta2 @ hidden_salida1.T )
        X02 = X02 + learning_rate * hidden_delta2
        W3 = W3 + learning_rate * (output_delta @ hidden_salida2.T)
        X03 = X03 + learning_rate * output_delta

    # ya recalcule las matrices de pesos
    # ahora avanzo la red, feed-forward
    # hidden_estimulos = W1 @ X.T + X01
    # hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
    # output_estimulos = W2 @ hidden_salidas + X02
    # output_salidas = func_eval_vec(output_FUNC, output_estimulos)
    
    hidden_salida1, hidden_salida2, output_salidas,  hidden_estimulo1, hidden_estimulo2 , hidden_ouput  = forward_propagation(X[fila:fila+1, :], W1, X01, W2, X02, W3, X03)
    

    # calculo el error promedio general de TODOS los X
    Error= np.mean( (Y.T - output_salidas)**2 )

    # tengo que hacer X01.T[0]  para que pase el vector
   # grafico.graficarVarias(W1, X01.T[0], epoch, -1)


# Evaluar el modelo en el conjunto de prueba
# loss, accuracy = model.evaluate(X_test, y_test)
# print("Accuracy on test set:", accuracy)

# Hacer predicciones con el modelo entrenado
# predictions = model.predict(new_data)
Error

0.09530823210793035

In [9]:
output_salidas

array([[-1.73194792e-14],
       [-1.77635684e-15],
       [ 2.66453526e-15],
       [ 1.11022302e-15],
       [ 2.66453526e-15],
       [ 3.77475828e-15],
       [ 1.33226763e-15],
       [-8.88178420e-16],
       [-1.55431223e-15],
       [ 3.77475828e-15],
       [ 5.77315973e-15],
       [-1.11022302e-15],
       [-4.21884749e-15],
       [ 1.77635684e-15],
       [ 2.22044605e-15],
       [-2.66453526e-15],
       [-4.88498131e-15],
       [-8.88178420e-16],
       [ 9.55106031e-01]])

## Nuevas Fotos

In [10]:
p_Fotos = root_path + "Nuevas_Fotos"
p_Fotos

'../Nuevas_Fotos'

In [12]:
#def Leer_fotos2(directorio_origen_f):
"""
    Lee la ruta lee interpretando que cada carpeta es una persona, y dentro de esa careta se encuentran las fotos de esa persona.
    Devuelve un diccionario con el nombre de la pesrona como índice y una lista de sus fotos.
"""

# Diccionario para almacenar las fotos
fotos_int= {} # diccionario interno q sea persona y el vector de fotos, 

# Recorrer las subcarpetas

for nombre_persona in os.listdir(p_Fotos):
    # Ruta de la subcarpeta
    ruta_persona = os.path.join(p_Fotos, nombre_persona)

    # Lista para almacenar las fotos de la persona
    fotos_persona = []    
    #convertir hiec a png
    # convertir_persona_HIEC_a_jpg(ruta_persona)
    # Recorrer las fotos de la persona
    for archivo in glob.glob(os.path.join(ruta_persona, "*.jpg")) :
        # Cargar la imagen
        imagen = cv2.imread(archivo)              
        fotos_persona.append(imagen)             
    fotos_int[nombre_persona] = fotos_persona
        
    #return fotos_int
fotos_int

{'Abel': [],
 'Carlos': [],
 'Federico G': [],
 'Federico R': [],
 'Franco S': [],
 'Gerard': [],
 'Gustavo': [],
 'Joaquin': [],
 'Lautaro': [],
 'Lisandro': [],
 'Matias': [],
 'Natalia': [],
 'Noelia': [],
 'Paola': [],
 'Victorio': []}

In [43]:
#from Funciones import  Leer_fotos
fotos=Leer_fotos2(p_Fotos)
fotos

{'Abel': [],
 'Carlos': [],
 'Federico G': [],
 'Federico R': [],
 'Franco S': [],
 'Gerard': [],
 'Gustavo': [],
 'Joaquin': [],
 'Lautaro': [],
 'Lisandro': [],
 'Matias': [],
 'Natalia': [],
 'Noelia': [],
 'Paola': [],
 'Victorio': []}

In [44]:
# Mostrar información
nombres_personas = list(fotos.keys())
print("Cantidad de personas:", len(nombres_personas))
print("Cantidad de fotos por persona:")
for nombre_persona, fotos_persona in fotos.items():
    print(f"- {nombre_persona}: {len(fotos_persona)}")

Cantidad de personas: 15
Cantidad de fotos por persona:
- Abel: 0
- Carlos: 0
- Federico G: 0
- Federico R: 0
- Franco S: 0
- Gerard: 0
- Gustavo: 0
- Joaquin: 0
- Lautaro: 0
- Lisandro: 0
- Matias: 0
- Natalia: 0
- Noelia: 0
- Paola: 0
- Victorio: 0


In [28]:
def recortar_imagen_C(image):
    import os
    import cv2
   # import matplotlib.pyplot as plt
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Utilizar un clasificador específico para caras
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

    # Detectar rostros en la imagen
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(30, 30))

    # Recorrer las caras detectadas
    face_images = []
    for (x, y, w, h) in faces:
        # Recortar la cara de la imagen
        face_images.append(image[y:y+h, x:x+w])

    return face_images

In [18]:
from Funciones import recortar_imagen

ImportError: cannot import name 'recortar_imagen' from 'Funciones' (c:\MaestriaDocs\DMA - Proyecto\caras_grupo4\scripts\Funciones.py)

In [ ]:
def matriz_fotos_nuevas(dir_name):
    import os
    # Tamaño fijo al que redimensionar todas las imágenes
    desired_size = (30, 30)
    # Listas para almacenar las imágenes y sus nombres
    images = [] #lista de fotos
    image_names = []
    image_person = [] #lista con los nombres de las personas de cada foto

    # Leer las imágenes del directorio y almacenarlas en las listas
    images = []
    for root, dirs, files in os.walk(dir_name_recorte):
        for dir_name in dirs:
            print("Carpeta:", dir_name)
            dir_path = os.path.join(root, dir_name) #directorio  de la persona

            for file_name in os.listdir(dir_path):

                if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                    image_path = os.path.join(dir_path, file_name)
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Leer en escala de grises
                if image is not None:
                    # Redimensionar la imagen al tamaño deseado
                    resized_image = cv2.resize(image, desired_size)

                    images.append(resized_image.flatten())  # Aplanar la imagen y agregarla a la lista
                    image_names.append(file_name)
                    image_person.append(dir_name)

    # Convertir la lista de imágenes a una matriz NumPy
    image_matrix = np.array(images) #matriz de fotos    
    return image_matrix

In [30]:
p_Foto ="C:\MaestriaDocs\DMA - Proyecto\caras_grupo4\Nuevas_fotos\Abel.jpeg"

import cv2 as cv
import numpy as np
from sklearn.decomposition import PCA
#from Funciones import recortar_imagen
# Cargar la imagen nueva


nueva_imagen = cv.imread(p_Foto)
if nueva_imagen is None:
    print ("no se pudo cargar la imagen")
else:
    #uso el proceso de guardar la imagen recortada
    face_images = recortar_imagen_C(nueva_imagen)
    # images.extend(face_images)

    for face_image in face_images:
        nueva_imagen = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)

    
        cv.imwrite('output.jpg', nueva_imagen)#probar que esta grabando

        # Redimensionar la imagen
        nueva_imagen = cv.resize(nueva_imagen, (30, 30))
        # Aplanar la imagen
        nueva_imagen = nueva_imagen.flatten()

        # Aplicar PCA
        lista=[]
        lista.append(nueva_imagen)
        image2_matrix = np.array(lista) 
#scaler = StandardScaler()
#lista=[]
#lista.append(image2_matrix)
#image2_matrix = scaler.fit_transform(lista)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 43-44: malformed \N character escape (1492511354.py, line 1)

In [ ]:
# Transformar la nueva imagen utilizando los componentes especificados
imagen_pca_reducida = transformar_imagen_pca(image2_matrix, scaler, pca_components, num_componentes, inicio_componente)

imagen_pca_reducida.shape

In [ ]:


prediction = model.predict(imagen_pca_reducida)

print("Predicción:", prediction)